<a href="https://colab.research.google.com/github/pavansai26/SEMANTIC-SEARCH-END-TO-PIPE-LINE-/blob/main/SEMANTIC_SEARCH_DEPLOYMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we need to install some libraries:


In [1]:
!pip install tensorflow
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.9 MB/s eta 0:00:00


Next, we can import the required libraries and load a pre-trained model:



In [2]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer

# Load pre-trained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Then, we can define a function to encode the query and documents using the BERT tokenizer and model:

In [3]:
def encode(query, documents):
    # Tokenize query
    query_tokens = tokenizer.encode_plus(
        query,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=256,
        return_attention_mask=True,
        return_tensors='tf'
    )

    # Tokenize documents
    document_tokens = tokenizer.batch_encode_plus(
        documents,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=256,
        return_attention_mask=True,
        return_tensors='tf'
    )

    # Encode query and documents using BERT model
    query_embedding = bert_model(query_tokens['input_ids'], query_tokens['attention_mask'])[1]
    document_embeddings = bert_model(document_tokens['input_ids'], document_tokens['attention_mask'])[1]

    return query_embedding, document_embeddings


We can then define a function to perform the semantic search using cosine similarity:



In [4]:
def semantic_search(query, documents):
    # Encode query and documents
    query_embedding, document_embeddings = encode(query, documents)

    # Compute cosine similarity between query and documents
    similarity = tf.keras.losses.cosine_similarity(query_embedding, document_embeddings)

    # Sort documents by similarity score in descending order
    sorted_indexes = tf.argsort(similarity, direction='DESCENDING').numpy()

    # Return sorted documents and their similarity scores
    results = [{'document': documents[i], 'similarity': similarity[i].numpy()} for i in sorted_indexes]

    return results


Finally, we can define a simple command-line interface to test our semantic search:



In [5]:
while True:
    query = input('Enter your query (or type "exit" to quit): ')
    if query == 'exit':
        break
    documents = [
        'This is the first document.',
        'This is the second document.',
        'This is the third document.'
    ]
    results = semantic_search(query, documents)
    print('Search results:')
    for result in results:
        print(f'{result["document"]}\nSimilarity score: {result["similarity"]}\n')


Enter your query (or type "exit" to quit): THIRD
Search results:
This is the third document.
Similarity score: -0.7636639475822449

This is the second document.
Similarity score: -0.8081446886062622

This is the first document.
Similarity score: -0.8082172870635986



KeyboardInterrupt: ignored

This will prompt the user to enter a query and display the search results for the given query. Here is an example output:



deploy the semantic search code using AWS EKS (Elastic Kubernetes Service) and Docker:

Dockerize the code:
First, create a Dockerfile in the project directory with the following contents:

In [ ]:
# Base image
FROM python:3.8-slim-buster

# Set working directory
WORKDIR /app

# Copy project files to working directory
COPY . .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Expose port
EXPOSE 5000

# Start server
CMD ["python", "app.py"]


The above Dockerfile uses the official Python 3.8 slim-buster base image, sets the working directory to /app, copies the project files to the working directory, installs the required dependencies, and starts the server on port 5000.

Next, build the Docker image using the following command:

In [ ]:
docker build -t semantic-search:latest .


Push Docker image to ECR:
Next, we need to push the Docker image to Amazon Elastic Container Registry (ECR) so that it can be deployed on AWS EKS.
First, create an ECR repository using the AWS Management Console or the AWS CLI:

In [ ]:
aws ecr create-repository --repository-name semantic-search


This will create a new ECR repository called "semantic-search". Next, authenticate the Docker CLI to your ECR registry and tag the Docker image:



In [ ]:
$(aws ecr get-login --no-include-email --region us-east-1)
docker tag semantic-search:latest <ECR repository URI>/semantic-search:latest


Replace <ECR repository URI> with the URI of your ECR repository. Finally, push the Docker image to ECR:



In [ ]:
docker push <ECR repository URI>/semantic-search:latest


Deploy the application on AWS EKS:
Now we can deploy the application on AWS EKS. First, create a Kubernetes deployment and service manifest file called "semantic-search.yaml" with the following contents:

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: semantic-search
spec:
  replicas: 3
  selector:
    matchLabels:
      app: semantic-search
  template:
    metadata:
      labels:
        app: semantic-search
    spec:
      containers:
      - name: semantic-search
        image: <ECR repository URI>/semantic-search:latest
        ports:
        - containerPort: 5000
---
apiVersion: v1
kind: Service
metadata:
  name: semantic-search
spec:
  selector:
    app: semantic-search
  ports:
  - name: http
    port: 80
    targetPort: 5000
  type: LoadBalancer


Replace <ECR repository URI> with the URI of your ECR repository. The above manifest file creates a Kubernetes deployment with 3 replicas and a Kubernetes service with a LoadBalancer type.

Next, deploy the application using the following command:

In [ ]:
kubectl apply -f semantic-search.yaml


This will create a deployment and service for the semantic search application on AWS EKS. To access the application, find the external IP address of the service by running the following command:

In [ ]:
kubectl get services semantic-search


This will display the external IP address of the service. You can then access the application by visiting the URL http://<external IP address>/ in your web browser.



